In [1]:
import numpy as np
from scipy import stats

from mayavi import mlab

In [2]:
"""
Helmoltz coils
==============

A script that computes the magnetic field generated by a pair of Helmoltz
coils.
"""

import numpy as np
from scipy import special, linalg

##############################################################################
# Function to caculate the field of a loop

def base_vectors(n):
    """ Returns 3 orthognal base vectors, the first one colinear to n.
    """
    # normalize n
    n = n / np.sqrt(np.square(n).sum(axis=-1))

    # choose two vectors perpendicular to n
    # choice is arbitrary since the coil is symetric about n
    if abs(n[0]) == 1 :
        l = np.r_[n[2], 0, -n[0]]
    else:
        l = np.r_[0, n[2], -n[1]]

    l = l / np.sqrt(np.square(l).sum(axis=-1))
    m = np.cross(n, l)
    return n, l, m


def B_field(r, n, r0, R):
    """
    returns the magnetic field from an arbitrary current loop calculated from
    eqns (1) and (2) in Phys Rev A Vol. 35, N 4, pp. 1535-1546; 1987.

    Parameters
    ----------
        n is normal vector to the plane of the loop at the center, current
            is oriented by the right-hand-rule.
        r is a position vector where the Bfield is evaluated:
            [x1 y2 z3 ; x2 y2 z2 ; ... ]
        r is in units of d
        r0 is the location of the center of the loop in units of d: [x y z]
        R is the radius of the loop

    Returns
    -------
        B is a vector for the B field at point r in inverse units of
    (mu I) / (2 pi d)
    for I in amps and d in meters and mu = 4 pi * 10^-7 we get Tesla
    """
    ### Translate the coordinates in the coil's frame
    n, l, m = base_vectors(n)

    # transformation matrix coil frame to lab frame
    trans = np.vstack((l, m, n))
    # transformation matrix to lab frame to coil frame
    inv_trans = linalg.inv(trans)

    r = r - r0	  #point location from center of coil
    r = np.dot(r, inv_trans) 	    #transform vector to coil frame

    #### calculate field

    # express the coordinates in polar form
    x = r[:, 0]
    y = r[:, 1]
    z = r[:, 2]
    rho = np.sqrt(x**2 + y**2)
    theta = np.arctan(x / y)
    # NaNs are generated where y is zero.
    theta[y == 0] = np.pi / 2

    E = special.ellipe((4 * R * rho)/( (R + rho)**2 + z**2))
    K = special.ellipk((4 * R * rho)/( (R + rho)**2 + z**2))
    dist = ((R - rho)**2 + z**2)
    Bz = 1 / np.sqrt((R + rho)**2 + z**2) * (
                K
              + E * (R**2 - rho**2 - z**2) / dist
              )
    Brho = z / (rho*np.sqrt((R + rho)**2 + z**2)) * (
               -K
              + E * (R**2 + rho**2 + z**2)/ dist
              )
    # On the axis of the coil we get a divided by zero here. This returns a
    # NaN, where the field is actually zero :
    Brho[dist == 0] = 0
    Brho[rho == 0] = 0
    Bz[dist == 0] = 0

    B = np.c_[np.cos(theta)*Brho, np.sin(theta)*Brho, Bz ]

    # Rotate the field back in the lab's frame
    B = np.dot(B, trans)
    return B


##############################################################################
# The grid of points on which we want to evaluate the field
X, Y, Z = np.mgrid[-0.15:0.15:31j, -0.15:0.15:31j, -0.15:0.15:31j]
# Avoid rounding issues :
f = 1e4  # this gives the precision we are interested in:
X = np.round(X * f) / f
Y = np.round(Y * f) / f
Z = np.round(Z * f) / f

# The (x, y, z) position vector
r = np.c_[np.ravel(X), np.ravel(Y), np.ravel(Z)]


##############################################################################
# The coil positions

# The center of the coil
r0  = np.r_[0, 0, 0.1]
# The normal to the coils
n  = np.r_[0, 0, 1]
# The radius
R  = 0.1

# Add the mirror image of this coils relatively to the xy plane :
r0 = np.vstack((r0, -r0 ))
R = np.r_[R, R]
n = np.vstack((n, n))	    # Helmoltz like configuration

##############################################################################
# Calculate field
# First initialize a container matrix for the field vector :
B = np.zeros_like(r)
# Then loop through the different coils and sum the fields :
for this_n, this_r0, this_R in zip(n, r0, R):
  this_n    = np.array(this_n)
  this_r0   = np.array(this_r0)
  this_R    = np.array(this_R)
  B += B_field(r, this_n, this_r0, this_R)



/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in divide
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in divide
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:81: RuntimeWarning: invalid value encountered in divide
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in divide
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in divide
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: divide by zero encountered in divide
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in add
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in multiply


In [1]:
B.shape

NameError: name 'B' is not defined

In [4]:
B.shape

(29791, 3)

In [3]:
from compute_field import B, X, Y, Z

compute_field.py:72: RuntimeWarning: divide by zero encountered in divide
  theta = np.arctan(x / y)
compute_field.py:72: RuntimeWarning: invalid value encountered in divide
  theta = np.arctan(x / y)
compute_field.py:81: RuntimeWarning: invalid value encountered in divide
  + E * (R**2 - rho**2 - z**2) / dist
compute_field.py:83: RuntimeWarning: divide by zero encountered in divide
  Brho = z / (rho*np.sqrt((R + rho)**2 + z**2)) * (
compute_field.py:83: RuntimeWarning: invalid value encountered in divide
  Brho = z / (rho*np.sqrt((R + rho)**2 + z**2)) * (
compute_field.py:85: RuntimeWarning: divide by zero encountered in divide
  + E * (R**2 + rho**2 + z**2)/ dist
compute_field.py:85: RuntimeWarning: invalid value encountered in add
  + E * (R**2 + rho**2 + z**2)/ dist
compute_field.py:85: RuntimeWarning: invalid value encountered in multiply
  + E * (R**2 + rho**2 + z**2)/ dist


In [6]:
X.shape

(31, 31, 31)

In [7]:
Y.shape

(31, 31, 31)

In [8]:
Z.shape

(31, 31, 31)

In [2]:
B.shape

NameError: name 'B' is not defined

In [9]:
# Data massaging

# Reshape the data to put it in a form that can be fed in Mayavi
Bx = B[:, 0]
By = B[:, 1]
Bz = B[:, 2]
Bx = np.reshape(Bx, X.shape)
By = np.reshape(By, X.shape)
Bz = np.reshape(Bz, X.shape)


In [10]:
Bnorm = np.sqrt(Bx**2 + By**2 + Bz**2)

# Threshold, to avoid the very high values
Bmax = stats.scoreatpercentile(Bnorm.ravel(), 99)

Bx[Bnorm > Bmax] = Bmax * (Bx / Bnorm)[Bnorm > Bmax]
By[Bnorm > Bmax] = Bmax * (By / Bnorm)[Bnorm > Bmax]
Bz[Bnorm > Bmax] = Bmax * (Bz / Bnorm)[Bnorm > Bmax]
Bnorm[Bnorm > Bmax] = Bmax

In [11]:
Bx.shape

(31, 31, 31)

In [12]:
Bnorm = np.sqrt(Bx**2 + By**2 + Bz**2)

# Threshold, to avoid the very high values
Bmax = stats.scoreatpercentile(Bnorm.ravel(), 99)

Bx[Bnorm > Bmax] = Bmax * (Bx / Bnorm)[Bnorm > Bmax]
By[Bnorm > Bmax] = Bmax * (By / Bnorm)[Bnorm > Bmax]
Bz[Bnorm > Bmax] = Bmax * (Bz / Bnorm)[Bnorm > Bmax]
Bnorm[Bnorm > Bmax] = Bmax

###############################################################################
# Visualization proper

# Create a mayavi figure black on white
mlab.figure(bgcolor=(0., 0., 0.), fgcolor=(1., 1., 1.), size=(640, 480))

# Create a vector_field: a data source that we can slice and dice
field = mlab.pipeline.vector_field(X, Y, Z, Bx, By, Bz,
                                   scalars=Bnorm,
                                   name='B field')
# Plot the vectors
vectors = mlab.pipeline.vectors(field,
                                scale_factor=abs(X[0, 0, 0] - X[1, 1, 1]),
                                colormap='hot')
mlab.axes()

# Mask 7 data points out of 8
vectors.glyph.mask_input_points = True
vectors.glyph.mask_points.on_ratio = 8

mlab.pipeline.vector_cut_plane(field, scale_factor=.1, colormap='hot')

# Add an iso_surface of the norm of the field
mlab.pipeline.iso_surface(mlab.pipeline.extract_vector_norm(field),
                          contours=[0.1*Bmax, 0.4*Bmax],
                          opacity=0.5, transparent=True)

mlab.view(28, 84, 0.71)
mlab.show()
#mlab.savefig('visualize_field.png')

/usr/local/lib/python2.7/site-packages/tvtk/array_handler.py:268: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [13]:
Bx.shape

(31, 31, 31)

In [14]:
X.shape

(31, 31, 31)